In [ ]:
# 🤖 Fairness-Focused ChatBot Training (Ultra-Reliable)

Train your chatbot with **fairness and politeness** as core values using a simple, conflict-free approach.

**What this does:**
- Trains on 42 fairness & politeness examples
- Creates ethical foundation that persists across personas
- Uses reliable, conflict-free installation
- Works on any GPU (T4, V100, etc.)

**Requirements:**
- Your `fairness_politeness_training.jsonl` file
- T4 GPU enabled in Colab
- About 20-40 minutes for training


In [ ]:
# Cell 1: Final Fix - Tokenizers Version Issue
print("🔄 Fixing tokenizers version conflict...")

# The issue is tokenizers version - let's fix it properly
print("🔧 Updating tokenizers to compatible version...")
!pip install --upgrade tokenizers>=0.19.0 --quiet

# Also ensure we have the right transformers version
print("📦 Ensuring transformers compatibility...")
!pip install --upgrade transformers>=4.44.0 --quiet

# Install remaining dependencies if needed
print("🔧 Installing any missing dependencies...")
!pip install accelerate>=0.33.0 datasets>=2.20.0 --quiet

print("✅ All packages updated!")

# Test imports with detailed error reporting
print("🧪 Testing imports...")
try:
    import torch
    print(f"✅ PyTorch {torch.__version__}")
    print(f"✅ CUDA available: {torch.cuda.is_available()}")
    
    # Test tokenizers first
    import tokenizers
    print(f"✅ Tokenizers {tokenizers.__version__}")
    
    from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
    print("✅ Transformers imported successfully")
    
    from datasets import Dataset
    print("✅ Datasets imported successfully")
    
    if torch.cuda.is_available():
        print(f"🎮 GPU: {torch.cuda.get_device_name()}")
        print(f"💾 GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    else:
        print("⚠️ No GPU detected. Please enable T4 GPU: Runtime → Change runtime type")
        
    print("🎉 All systems ready for fairness training!")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    
    # If still failing, try the nuclear option
    print("🚨 Trying nuclear option - complete reinstall...")
    !pip install --force-reinstall --no-cache-dir transformers tokenizers accelerate datasets --quiet
    
    try:
        from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
        from datasets import Dataset
        print("✅ Nuclear option worked! Ready for training!")
    except:
        print("💡 Please restart runtime (Runtime → Restart runtime) and run this cell again")
        print("💡 Sometimes Colab needs a fresh start")
    
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    print("💡 Try restarting runtime if issues persist")


In [ ]:
# Cell 2: Load Model (Simple & Reliable)
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

print("🚀 Loading model for fairness training...")

# Use GPT-2 Medium - reliable, well-tested, good for conversation
model_name = "gpt2-medium"
print(f"📦 Loading {model_name}...")

try:
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    print("✅ Tokenizer loaded")
    
    # Load model with appropriate settings
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None,
        low_cpu_mem_usage=True
    )
    
    print("✅ Model loaded successfully!")
    print(f"📊 Model parameters: {model.num_parameters():,}")
    print(f"💾 Model size: ~{model.num_parameters() * 2 / 1e9:.1f} GB")
    print("🎯 Ready for fairness training!")
    
except Exception as e:
    print(f"❌ Model loading failed: {e}")
    print("💡 Make sure the previous installation completed successfully")
    print("💡 Try restarting runtime if issues persist")


In [ ]:
# Cell 3: Upload and Prepare Fairness Training Data
from google.colab import files
import json
from datasets import Dataset

print("📁 Upload your fairness_politeness_training.jsonl file:")
print("   This file contains 42 examples teaching fairness and politeness")
print("   Location: fine_tuning/data/fairness_politeness_training.jsonl")
print()

# Upload the file
uploaded = files.upload()

# Load and validate the data
filename = list(uploaded.keys())[0]
fairness_data = []

try:
    with open(filename, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            if line.strip():  # Skip empty lines
                try:
                    data = json.loads(line.strip())
                    # Validate required fields
                    if all(key in data for key in ['instruction', 'input', 'output']):
                        fairness_data.append(data)
                    else:
                        print(f"⚠️ Line {line_num}: Missing required fields")
                except json.JSONDecodeError:
                    print(f"⚠️ Line {line_num}: Invalid JSON format")
    
    print(f"✅ Loaded {len(fairness_data)} valid fairness training examples")
    
    # Show sample data
    if fairness_data:
        print(f"\n🔍 Sample training example:")
        sample = fairness_data[0]
        print(f"Instruction: {sample['instruction']}")
        print(f"Output: {sample['output'][:100]}...")
        
except Exception as e:
    print(f"❌ Error loading data: {e}")
    print("💡 Make sure your file is in JSONL format")

# Format data for GPT-2 training
def format_for_training(examples):
    texts = []
    for instruction, input_text, output in zip(examples["instruction"], examples["input"], examples["output"]):
        # Create a clear conversational format
        if input_text.strip():
            text = f"Human: {instruction}\n{input_text}\nAssistant: {output}<|endoftext|>"
        else:
            text = f"Human: {instruction}\nAssistant: {output}<|endoftext|>"
        texts.append(text)
    return {"text": texts}

# Convert to dataset
dataset = Dataset.from_list(fairness_data)
dataset = dataset.map(format_for_training, batched=True)

print(f"\n🎯 Dataset prepared with {len(dataset)} examples!")
print("\n📋 Your model will learn:")
print("   • Fair treatment of all users regardless of background")
print("   • Polite responses even to rude or demanding users")
print("   • Balanced perspectives on controversial topics")
print("   • Empathy and cultural sensitivity")
print("   • Constructive conflict resolution")
print("   • Respectful boundary setting")


In [ ]:
# Cell 4: Train the Model on Fairness Data (FP32 Stable Version)
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

print("🏋️ Setting up fairness training...")

# Ensure model is in FP32 mode for stability
print("🔧 Converting model to FP32 for stable training...")
model = model.float()  # Force FP32
if torch.cuda.is_available():
    model = model.cuda()

print("✅ Model ready in FP32 mode")

# Tokenize the dataset
def tokenize_function(examples):
    # Tokenize with proper truncation and padding
    return tokenizer(
        examples["text"], 
        truncation=True, 
        padding=True, 
        max_length=512,  # Reasonable length for conversations
        return_tensors="pt"
    )

print("🔄 Tokenizing dataset...")
tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=dataset.column_names,
    desc="Tokenizing"
)

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Causal language modeling (not masked)
    pad_to_multiple_of=None  # Disable padding optimization for stability
)

# Training configuration optimized for fairness learning (Stable Version)
training_args = TrainingArguments(
    output_dir="./fairness_training_output",
    overwrite_output_dir=True,
    
    # Training schedule
    num_train_epochs=3,  # Full training for better fairness learning
    per_device_train_batch_size=1,  # Reduced for stability
    gradient_accumulation_steps=8,  # Maintain effective batch size of 8
    
    # Learning rate and optimization
    learning_rate=3e-5,  # Slightly lower for stability
    warmup_steps=5,  # Reduced for small dataset
    weight_decay=0.01,
    
    # Logging and saving
    logging_steps=2,  # More frequent logging for small dataset
    save_strategy="epoch",
    eval_strategy="no",
    
    # Stability optimizations (Fixed FP16 issue)
    fp16=False,  # Disabled FP16 to avoid gradient scaling issues
    bf16=False,  # Also disable bf16 for maximum stability
    dataloader_drop_last=True,
    remove_unused_columns=False,
    
    # Disable wandb logging to avoid distractions
    report_to=[],  # No logging to wandb
    
    # Reproducibility
    seed=42,
    data_seed=42,
)

# Create the trainer with stability settings
print("🔧 Creating stable trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
    processing_class=tokenizer,  # Updated parameter name
)

print("🚀 Starting fairness & politeness training...")
print(f"   📊 Training on {len(tokenized_dataset)} examples")
print(f"   🔄 {training_args.num_train_epochs} epochs")
print(f"   ⏱️ Estimated time: 10-20 minutes (FP32 mode)")
print(f"   📈 Watch the loss decrease as the model learns fairness!")
print()

# Clear any cached gradients and start fresh
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Start training with error handling
try:
    print("🎯 Beginning stable FP32 training...")
    trainer_stats = trainer.train()
    print("✅ Fairness training completed successfully!")
    print(f"📉 Final training loss: {trainer_stats.training_loss:.4f}")
    
    # Clear cache after training
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
except RuntimeError as e:
    if "FP16" in str(e) or "unscale" in str(e):
        print(f"❌ FP16 error persists: {e}")
        print("🔄 Trying emergency FP32 fix...")
        
        # Emergency fix: recreate everything in FP32
        model = model.float()
        training_args.fp16 = False
        training_args.bf16 = False
        
        trainer = Trainer(
            model=model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=tokenized_dataset,
            processing_class=tokenizer,
        )
        
        try:
            trainer_stats = trainer.train()
            print("✅ Emergency fix worked! Training completed!")
            print(f"📉 Final training loss: {trainer_stats.training_loss:.4f}")
        except Exception as e2:
            print(f"❌ Emergency fix failed: {e2}")
            print("💡 Please restart runtime and try again")
    else:
        print(f"❌ Training failed: {e}")
        print("💡 Try restarting runtime if issues persist")
        
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    print("💡 Try restarting runtime")


In [ ]:
# Cell 5: Test Your Fairness-Trained Model
import torch

print("🧪 Testing your fairness-trained chatbot...")

def generate_response(prompt, max_length=150, temperature=0.7):
    """Generate a response using the fairness-trained model"""
    # Format the prompt properly
    formatted_prompt = f"Human: {prompt}\nAssistant:"
    
    # Tokenize
    inputs = tokenizer.encode(formatted_prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = inputs.cuda()
    
    # Generate with good parameters for conversation
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=len(inputs[0]) + max_length,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            top_k=50,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1
        )
    
    # Decode and clean up
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace(formatted_prompt, "").strip()
    
    # Clean up any remaining artifacts
    if response.startswith("Assistant:"):
        response = response[10:].strip()
    
    return response

# Test scenarios to verify fairness training
test_scenarios = [
    "You're being really unhelpful and stupid!",
    "I don't trust people from other countries. What do you think?",
    "Can you help me write a complaint letter about poor service?",
    "I'm feeling really frustrated with my situation right now.",
    "What's your opinion on this controversial political topic?"
]

print("🎯 Testing fairness and politeness responses:\n")

for i, scenario in enumerate(test_scenarios, 1):
    print(f"Test {i}: {scenario}")
    response = generate_response(scenario)
    print(f"Response: {response}")
    print("-" * 60)

print("\n✅ Fairness testing complete!")
print("\n🎉 Your model should now demonstrate:")
print("   • Polite responses to rude comments")
print("   • Fair treatment regardless of user background")
print("   • Balanced perspectives on sensitive topics")
print("   • Empathetic and helpful communication")
print("   • Professional boundary setting")

print("\n💾 Next steps:")
print("   1. Save your model (next cell)")
print("   2. Download for use in your Next.js app")
print("   3. Test with your actual chatbot personas")
print("   4. The fairness training will work alongside any persona!")


In [ ]:
# Cell 6: Save and Download Your Fairness-Trained Model
import os
from google.colab import files

print("💾 Saving your fairness-trained chatbot model...")

# Save the model and tokenizer
save_directory = "./fairness_chatbot_model"
os.makedirs(save_directory, exist_ok=True)

try:
    # Save model and tokenizer
    model.save_pretrained(save_directory)
    tokenizer.save_pretrained(save_directory)
    
    print("✅ Model saved successfully!")
    print(f"📁 Saved to: {save_directory}")
    
    # Create a simple inference script for your Next.js app
    inference_script = '''# Fairness-Trained ChatBot Inference Script
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

class FairnessChatBot:
    def __init__(self, model_path):
        """Load the fairness-trained model"""
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        
    def generate_response(self, prompt, max_length=150, temperature=0.7):
        """Generate a fair and polite response"""
        formatted_prompt = f"Human: {prompt}\\nAssistant:"
        
        inputs = self.tokenizer.encode(formatted_prompt, return_tensors="pt")
        if torch.cuda.is_available():
            inputs = inputs.cuda()
        
        with torch.no_grad():
            outputs = self.model.generate(
                inputs,
                max_length=len(inputs[0]) + max_length,
                temperature=temperature,
                do_sample=True,
                top_p=0.9,
                top_k=50,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                repetition_penalty=1.1
            )
        
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = response.replace(formatted_prompt, "").strip()
        
        if response.startswith("Assistant:"):
            response = response[10:].strip()
            
        return response

# Usage example:
# chatbot = FairnessChatBot("./fairness_chatbot_model")
# response = chatbot.generate_response("Hello, how can you help me today?")
# print(response)
'''
    
    # Save the inference script
    with open(f"{save_directory}/inference.py", "w") as f:
        f.write(inference_script)
    
    print("📝 Created inference.py for easy integration")
    
    # Create a README for the model
    readme_content = '''# Fairness-Trained ChatBot Model

This model has been fine-tuned on 42 examples of fair and polite conversation patterns.

## What it learned:
- Fair treatment of all users regardless of background
- Polite responses even to rude or demanding users  
- Balanced perspectives on controversial topics
- Empathy and cultural sensitivity
- Constructive conflict resolution
- Respectful boundary setting

## How to use:
1. Load the model using the provided inference.py script
2. The fairness training works as a foundation layer
3. You can still use personas - they'll be fair and polite
4. Perfect for customer service, education, or general chat

## Integration with Next.js:
- Replace your Gemini API calls with local inference
- Use the FairnessChatBot class from inference.py
- The model will maintain fairness across all personas

## Model Details:
- Base: GPT-2 Medium (355M parameters)
- Training: 3 epochs on fairness data
- Format: Conversational (Human/Assistant)
- Size: ~1.4GB
'''
    
    with open(f"{save_directory}/README.md", "w") as f:
        f.write(readme_content)
    
    print("📖 Created README.md with usage instructions")
    
    # Zip the model for download
    print("🗜️ Creating download package...")
    !zip -r fairness_chatbot_model.zip {save_directory}
    
    print("📥 Downloading your fairness-trained model...")
    files.download("fairness_chatbot_model.zip")
    
    print("\n🎉 SUCCESS! Your fairness-trained chatbot is ready!")
    print("\n📋 What you got:")
    print("   ✅ Trained model files (pytorch_model.bin, config.json)")
    print("   ✅ Tokenizer files")
    print("   ✅ inference.py - Ready-to-use Python class")
    print("   ✅ README.md - Complete usage guide")
    
    print("\n🚀 Next steps for your Next.js app:")
    print("   1. Extract the zip file")
    print("   2. Set up Python inference server or use directly")
    print("   3. Replace Gemini API calls with your trained model")
    print("   4. Test with your existing personas")
    print("   5. Enjoy fair and polite conversations!")
    
    print("\n💡 Pro tip: The fairness training is now the foundation.")
    print("   Any persona you create will automatically be fair and polite!")
    
except Exception as e:
    print(f"❌ Error saving model: {e}")
    print("💡 Make sure training completed successfully")
    print("💡 Check available disk space")


In [ ]:
# Cell 7: Continue Training for Lower Loss (Optional)
print("🔄 Want to decrease loss further? Let's continue training!")

# Continue training for 2 more epochs
extended_training_args = TrainingArguments(
    output_dir="./fairness_training_extended",
    overwrite_output_dir=True,
    
    # Extended training
    num_train_epochs=2,  # Additional epochs
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    
    # Slightly lower learning rate for fine-tuning
    learning_rate=1e-5,  # Lower for continued training
    warmup_steps=2,
    weight_decay=0.01,
    
    # Logging
    logging_steps=2,
    save_strategy="epoch",
    eval_strategy="no",
    
    # Stability
    fp16=False,
    bf16=False,
    dataloader_drop_last=True,
    remove_unused_columns=False,
    report_to=[],
    
    seed=42,
    data_seed=42,
)

# Create extended trainer
extended_trainer = Trainer(
    model=model,  # Continue from current model
    args=extended_training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
    processing_class=tokenizer,
)

print("🚀 Continuing training with lower learning rate...")
print("   📈 Target: Get loss below 2.5")
print("   ⏱️ Additional time: ~10 minutes")

try:
    # Clear cache and continue
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    extended_stats = extended_trainer.train()
    print("✅ Extended training completed!")
    print(f"📉 New final loss: {extended_stats.training_loss:.4f}")
    
    # Calculate improvement
    original_loss = 2.743600
    new_loss = extended_stats.training_loss
    improvement = ((original_loss - new_loss) / original_loss) * 100
    print(f"🎯 Loss improvement: {improvement:.1f}%")
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
except Exception as e:
    print(f"❌ Extended training failed: {e}")
    print("💡 Your original training is still good - proceed to testing!")


In [ ]:
# 🦥 Fairness-Focused ChatBot Training with Unsloth

Train your chatbot with **fairness and politeness** as core values that persist across all personas.

**What you'll need:**
- Your `fairness_politeness_training.jsonl` file (42 examples)
- T4 GPU enabled in Colab
- About 30-60 minutes for training


In [ ]:
# FIXED Installation - handles CUDA compatibility issues
%%capture
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Check if we're on Colab and have GPU
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    major_version, minor_version = torch.cuda.get_device_capability()
    print(f"CUDA capability: {major_version}.{minor_version}")
else:
    print("⚠️ No GPU detected. Please enable T4 GPU in Runtime > Change runtime type")

# Install Unsloth with proper error handling
try:
    import subprocess
    import sys
    
    # Install Unsloth
    subprocess.check_call([sys.executable, "-m", "pip", "install", "unsloth[colab-new]", "@", "git+https://github.com/unslothai/unsloth.git"])
    
    # Install dependencies based on GPU capability
    if torch.cuda.is_available():
        major_version, minor_version = torch.cuda.get_device_capability()
        if major_version >= 8:
            # New GPUs
            subprocess.check_call([sys.executable, "-m", "pip", "install", "--no-deps", "packaging", "ninja", "einops", "xformers", "trl", "peft", "accelerate", "bitsandbytes"])
        else:
            # Older GPUs (T4, V100)
            subprocess.check_call([sys.executable, "-m", "pip", "install", "--no-deps", "xformers", "trl", "peft", "accelerate", "bitsandbytes"])
    
    print("✅ Installation successful!")
    
except Exception as e:
    print(f"❌ Installation failed: {e}")
    print("💡 Try restarting runtime and running again")


In [ ]:
# Load model with error handling
try:
    from unsloth import FastLanguageModel
    import torch
    
    print("🚀 Loading model...")
    
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    
    print("⚡ Adding LoRA adapters...")
    
    # Add LoRA adapters
    model = FastLanguageModel.get_peft_model(
        model,
        r = 16,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_alpha = 16,
        lora_dropout = 0,
        bias = "none",
        use_gradient_checkpointing = "unsloth",
        random_state = 3407,
    )
    
    print("✅ Model loaded successfully!")
    
except Exception as e:
    print(f"❌ Model loading failed: {e}")
    print("💡 Make sure the previous installation cell completed successfully")


In [ ]:
# Upload your fairness training data
from google.colab import files
import json
from datasets import Dataset

print("📁 Upload your fairness_politeness_training.jsonl file:")
print("   (This file contains 42 examples teaching fairness and politeness)")
uploaded = files.upload()

# Load the data
filename = list(uploaded.keys())[0]
fairness_data = []
with open(filename, 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():  # Skip empty lines
            fairness_data.append(json.loads(line.strip()))

print(f"✅ Loaded {len(fairness_data)} fairness training examples")

# Show a sample
if fairness_data:
    print(f"\n🔍 Sample example:")
    print(f"Instruction: {fairness_data[0]['instruction']}")
    print(f"Output: {fairness_data[0]['output'][:100]}...")

# Format for training
def format_prompts(examples):
    texts = []
    for instruction, input_text, output in zip(examples["instruction"], examples["input"], examples["output"]):
        text = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{output}<|eot_id|><|end_of_text|>"
        texts.append(text)
    return {"text": texts}

dataset = Dataset.from_list(fairness_data)
dataset = dataset.map(format_prompts, batched=True)
print(f"🎯 Dataset ready with {len(dataset)} examples for fairness training!")


In [ ]:
# Train the model on fairness data
from trl import SFTTrainer, SFTConfig
import torch

print("🏋️ Setting up training...")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 3,  # Full training for better fairness learning
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

print("🚀 Starting fairness & politeness training...")
print("   This will take about 30-60 minutes...")
trainer_stats = trainer.train()
print("✅ Training completed!")


In [ ]:
# Test your fairness-trained model
FastLanguageModel.for_inference(model)

def test_model(question):
    inputs = tokenizer(
        f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n", 
        return_tensors="pt"
    ).to("cuda")
    
    outputs = model.generate(**inputs, max_new_tokens=200, temperature=0.7, do_sample=True)
    response = tokenizer.batch_decode(outputs)[0]
    
    # Extract just the assistant's response
    if "<|start_header_id|>assistant<|end_header_id|>" in response:
        assistant_response = response.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
        assistant_response = assistant_response.split("<|eot_id|>")[0].strip()
        return assistant_response
    return response

# Test with fairness scenarios
test_questions = [
    "You're being too slow and unhelpful!",
    "Can you tell me which political party is better?",
    "I think people from certain countries are lazy",
    "I'm really frustrated and need help",
    "Can you insult my annoying coworker?"
]

print("🧪 Testing your fairness-trained chatbot:\n")
for question in test_questions:
    print(f"❓ User: {question}")
    print(f"🤖 Bot: {test_model(question)}")
    print("-" * 60)


In [ ]:
# Save and download your fairness-trained model
print("💾 Saving your fairness-trained model...")

# Save for production use
model.save_pretrained_merged("fairness_chatbot", tokenizer, save_method="merged_16bit")

# Also save GGUF format for local use
model.save_pretrained_gguf("fairness_chatbot_gguf", tokenizer, quantization_method="q4_k_m")

# Create download package
!zip -r fairness_chatbot_model.zip fairness_chatbot/

print("🎉 Your fairness-focused chatbot is ready!")
print("📊 Trained on 42 fairness & politeness examples")
print("🎯 Core values: Fair treatment, respectful communication, empathy")
print("🔄 Ready for persona integration while maintaining ethical foundation")
print("📥 Downloading...")

from google.colab import files
files.download('fairness_chatbot_model.zip')
print("✅ Download complete!")
